# Training with Zwift

I enjoy road biking. It's one of the few hobbies that's remained throughout my life. In elementary school, I rode my bike in my town's annual parade. As a college freshman, I salvaged a '70s Univega road bike ("Maria"), building her up from the frame. We completed our first century ride in 2015. That magnificent bike was stolen while I was in grad school, plucked off the porch at a BBQ. Maria was replaced by my current bike, a [2017 Fuji Touring Bike](https://www.cyclingabout.com/2017-fuji-touring-bike/) ("björn"). Björn has never failed me. 

Most of my riding over the past year has been indoors, largely due to the COVID pandemic and rehabilitation following a shoulder surgery. I joined [Zwift](https://us.zwift.com/) last year on the recommendation of a friend, and have thoroughly enjoyed it. I'm currently using Zwift to train for an extended bikepack trip next summer. Our rough plan is to start in Astoria, OR and continue [~1400 miles along the Pacific Coast](https://www.adventurecycling.org/routes-and-maps/adventure-cycling-route-network/interactive-network-map/) to San Diego, CA. 

One of Zwift's best features is their ability to create beautiful and intuitive figures from user data, acquired from a bluetooth enabled bike trainer. Due to thoughtful data access options, I learned today that I can dive into my ride data as well.<br><br>

**GOAL:** *(1)* Characterize bike training patterns over time, starting from March 1, 2022; and *(2)* describe anticipated level of functioning on May 1, 2023.<br>
**DATA:** Ride data downloaded from personal Zwift account.<br>
**ANALYSIS:** Exploratory data analysis; Bayesian modeling.<br>
**ETHICAL CONSIDERATIONS:** There are no apparent issues with transparency, accountability, or equity in terms of avaiable data. To avoid any unforseen privacy issues, I will not be posting the raw ride data on Github. I will do my best to characterize the data in this Notebook to justify any insights drawn from the analyses.<br>
**ADDITIONAL CONSIDERATIONS:** None.<br>

## Load libraries

In [ ]:
import os

# data wrangling/analysis
import fitdecode
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

Check out [this tutorial](https://towardsdatascience.com/parsing-fitness-tracker-data-with-python-a59e7dc17418) on how to parse .fit files (downloaded from Zwift) using the [`fitdecode`](https://pypi.org/project/fitdecode/) library.